# Tutorial: Connecting to a local Memgraph Docker container

This notebook will guide you through setting up a local workflow using a Memgraph Docker container.

## Part 1: Establishing a connection

In [1]:
from dtgraph import Neo4jGraph, Rule, Transformation

In the following, we will connect to a [Memgraph](https://memgraph.com/) instance running inside a Docker container.
You can type `sudo docker run --name memgraph2.14.0 -p 7687:7687 -p 7444:7444 -p 3000:3000 memgraph/memgraph-platform:2.14.0-memgraph2.14.0-lab2.11.1` to install and run Memgraph locally. (Of course you need to have [Docker](https://docs.docker.com/engine/install/)  already installed on your system.) You should then be able to access [Memgraph lab](http://localhost:3000/) running locally on your computer. (This is the equivalent of Neo4j browser.)

*Note:* We have specifically tested the compatibility of this framework with Memgraph 2.14.0 and Memgraph lab 2.11.1, which were the latest versions by the time of writting this guide. 

The default configuration of Memgraph does not use authentication. Make sure to use *database=memgraph* to indicate to the backend that we use Memgraph and not Neo4j. This makes a difference because the syntax for creating indexes is different compared to with Neo4j.

In [2]:
hostname = "localhost"
password = ""

In [3]:
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, database="memgraph", username="", password=password)

You can check if it already contains content:

In [4]:
graph.output_all_nodes()

Info: There is currently 171 node(s) in the database.


## Part II: (Optional) Loading a curated scenario

If your database is empty or you don't have it loaded with the *Movie dataset*, you can import it using the following command:

In [5]:
from dtgraph.scenarios.movies import Movies
Movies.load(graph)

Flushed database: Deleted 171 nodes, deleted 253 relationships, completed after None ms.
Load scenario: Added 171 labels, created 171 nodes, set 0 properties, created 253 relationships, completed after None ms.


## Part III: Defining and executing a rule

Then you can write your own transformation rule:

In [6]:
my_rule = Rule('''
        MATCH (n:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(o:Person)
        => 
        (x = (n) : Actor {
            name = n.name,
            born = n.born
        })-[(m) : COLLEAGUE {
            movie = m.title
        }]->(y = (o) : Actor {
            name = o.name,
            born = o.born
        })
''')

And execute it after after wrapping it in a Transformation object:

In [7]:
my_transform = Transformation([my_rule])
my_transform.apply_on(graph)

ClientError: {code: Memgraph.ClientError.MemgraphError.MemgraphError} {message: line 2:22 no viable alternative at input 'CREATEINDEXidx_dummy'}

Yay, 102 nodes and 204 labels have been created! You can check with the following query on your Neo4j browser the result (alongside the initial dataset, for now):
```
MATCH (n)
RETURN n
```

The line `Index: Added 1 index, completed after 7 ms.` indicates that an index has been added. This is part of the internals of the library to speed up the computation of the output graph.

We can now add a new rule to this transformation:

In [ ]:
my_second_rule = Rule('''
    MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person)
    =>
    (x = (d) : Director {
        name = d.name,
        born = d.born
    })-[(m) : SUPERVISED {
        movie = m.title
    }]->(y = (a) : Actor {
        name = a.name,
        born = a.born
    })
''')
my_transform.add(my_second_rule)

We see that 23 nodes and 51 labels have been added.

Let us now investigate the output of the transformation. We have two rules and both extract `Person` nodes. The first rule creates nodes only of type `Actor` while the second one also creates nodes of type `Director`.
We can confirm with the following query that a single node is created on the output with both labels if a person is both an Actor and a Director of some films:

```
MATCH (n)
WHERE n:Actor and n:Director
RETURN n
```

This query should return the following output:
```
╒══════════════════════════════════════════════════════════════════════╕
│n                                                                     │
╞══════════════════════════════════════════════════════════════════════╡
│(:Actor:Director:_dummy {born: 1967,name: "James Marshall",_id: "(4:7f│
│732a8b-14ba-4846-8477-f326f7a1b5d0:2469)"})                           │
├──────────────────────────────────────────────────────────────────────┤
│(:Actor:Director:_dummy {born: 1956,name: "Tom Hanks",_id: "(4:7f732a8│
│b-14ba-4846-8477-f326f7a1b5d0:2515)"})                                │
├──────────────────────────────────────────────────────────────────────┤
│(:Actor:Director:_dummy {born: 1930,name: "Clint Eastwood",_id: "(4:7f│
│732a8b-14ba-4846-8477-f326f7a1b5d0:2543)"})                           │
├──────────────────────────────────────────────────────────────────────┤
│(:Actor:Director:_dummy {born: 1944,name: "Danny DeVito",_id: "(4:7f73│
│2a8b-14ba-4846-8477-f326f7a1b5d0:2586)"})                             │
├──────────────────────────────────────────────────────────────────────┤
│(:Actor:Director:_dummy {born: 1942,name: "Werner Herzog",_id: "(4:7f7│
│32a8b-14ba-4846-8477-f326f7a1b5d0:2503)"})                            │
└──────────────────────────────────────────────────────────────────────┘
```

This is indeed the desired behavior to be able to define the content of new nodes with multiple queries, independent of each other.

Finally, `.eject()` removes the internal bookeeping data and let the output of the transformation alongside of the initial data.

In [ ]:
my_transform.eject()

Note that the index has been removed from the database.